In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import csv
import pandas as pd
import json
import time
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
import string 
import unicodedata as ud
from greek_stemmer import GreekStemmer
import pymongo
import numpy as np
import re
import networkit as nk


def tokenize(row):
    return word_tokenize(row)

def preprocess_doc(doc: str) -> list:
    stemmer = GreekStemmer()
    d = {ord('\N{COMBINING ACUTE ACCENT}'):None}
    # load stopwords
    with open('stopwords.txt', encoding='utf-8') as file:
        stopwords = [line.rstrip() for line in file]
    #doc = doc.lower()
    if doc!="":
        words = [stemmer.stem(ud.normalize('NFD',w).upper().translate(d)).lower() for w in filter(None, re.split("[,~`; _'.\-!?:]+",doc)) if w not in stopwords and w not in string.punctuation]
    return words

def create_index(dataframe):
    chunk = []
    counter = 0
    start_time = time.time()
    
    for data in dataframe:
        tokens = {}
        chunk = (data["speech"].values.tolist())
        print("Length of chunk: ", len(chunk))
        #chunk = ["This is a sentance","This is another one"]
        for i, row in enumerate(chunk):
            words_in_row = preprocess_doc(row.lower())
            '''
            
            TOKENIZED ROWS IN ROW
            We now need to remove stopwords, perform stemming and remove punctuation
            Afterwards, we can add them to the index. This will be done for each row of the dataframe chunk.
            When the chunk is finished, we will create a json file with the index of this chunk and we'll do the process again for the next chunk.
            
            '''
            
            for word in words_in_row:
                
                if word in tokens.keys():
                    #term already in index
                    if i not in tokens[word]["postinglist"].keys():
                        #term in other document
                        #n = tokens[word]["numdoc"]
                        tokens[word]["postinglist"][str(i)] = words_in_row.count(word)
                        tokens[word]["numdoc"] = len(tokens[word]["postinglist"])

                    else:
                        #term in same document
                        n = tokens[word]["numdoc"]
                        tokens[word]["numdoc"] = n+1
                        pass
                        
                else:
                    #term not in index
                    tokens[word]={"numdoc":1, "postinglist":{str(i) : words_in_row.count(word)}}
                
                
        
        counter +=1
        print("CHUNK", counter, " FINISHED")
        print("Number of Tokens: ", len(tokens))
        
        print(type(tokens))
        if(counter == 1):
            break
    
    
    return tokens

In [121]:
import igraph as ig
def has_node(graph, name):
    try:
        graph.vs.find(name=name)
    except:
        return False
    return True

def create_index(dataframe):
    chunk = []
    counter = 0
    start_time = time.time()
    
    for data in dataframe:
        tokens = {}
        chunk = (data["speech"].values.tolist())
        print("Length of chunk: ", len(chunk))
        #chunk = ["This is a sentance","This is another one"]
        #1 row is a document
        graphs = {}
        for i, row in enumerate(chunk):
            words_in_row = preprocess_doc(row.lower())
            graph = nk.Graph(len(words_in_row), weighted=True, directed=True)
            att=graph.attachNodeAttribute('name',str)
            print('WORDS IN DOC: ', (words_in_row)) 
            for j,word in enumerate(words_in_row):
                att[j]=word
                print(graph.name(j))
                k=1
                while(k<4 and k+j<len(words_in_row)):
                    if(graph.hasEdge(j, k+j)==False):
                        graph.addEdge(j, k+j)
                    else:
                        new_weight = graph.weight(j,k+j) + 1
                        print('updating weight')
                        graph.increaseWeight(j, j+k, new_weight)
                    k+=1
                    
            graphs[i]=graph
            print(row)
            break
                
                
        
        counter +=1
        print("CHUNK", counter, " FINISHED")
        print("Number of Tokens: ", len(tokens))
        
        print(type(tokens))
        if(counter == 1):
            break
    
    
    return tokens, graphs

In [122]:
dataframe = pd.read_csv('Greek_Parliament_Proceedings_1989_2020.csv', chunksize=1000)
index, graphs = create_index(dataframe)
print(graphs[0])

Length of chunk:  1000
WORDS IN DOC:  ['παρακαλειτα', 'γραμματ', 'βουλγαρακ', 'συνοδευς', 'μακαρι', 'αρχιεπισκοπ', 'αθην', 'πας', 'ελλαδ', 'σεραφειμ', 'συνοδευοντ', 'αυτον', 'μελ', 'ιερ', 'συνοδ', 'κατα', 'εισοδ', 'αιθους', 'βουλ', 'προκειμεν', 'τελεσθ', 'αγιασμ', 'συνεχει', 'τελειτα', 'καθιερωμεν', 'αγιασμ']


AttributeError: 'networkit.graph.Graph' object has no attribute 'name'

In [120]:
g=graphs[0]
g.

In [84]:
graph = graphs[0]
graph.write_svg('/Users/harry/a.svg')

In [94]:
import networkit as nk
